# Database Exploration Notebook

This notebook provides tools for exploring the symbology database using pandas.
Use this to find companies with missing fields, abnormal filing results, and other data quality issues.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import warnings

# Import project-specific modules
import sys
import os
sys.path.append('/home/steven/symbology/src')

from utils.config import settings
from database.base import init_db, get_db_session
from database.utils import (
    explore_table_schema,
    analyze_missing_data,
    find_duplicates,
    get_companies_with_missing_fields,
    analyze_companies_data_quality,
    analyze_filing_patterns,
    get_data_quality_summary,
    quick_query,
    search_companies,
    get_company_details,
    plot_missing_data_heatmap,
    get_counts_for_companies
)

warnings.filterwarnings('ignore')
plt.style.use('default')

print("Libraries imported successfully!")

In [ ]:
# Initialize database connection
engine, session = init_db(settings.database.url)

# Create pandas-friendly engine for direct SQL queries
pd_engine = create_engine(settings.database.url)

print(f"Connected to database: {settings.database.database_name}")
print(f"Host: {settings.database.host}:{settings.database.port}")

# Test connection
with pd_engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    version = result.fetchone()[0]
    print(f"PostgreSQL version: {version[:50]}...")

## Database Schema Exploration

Let's start by exploring the database structure to understand what tables and columns are available.

In [ ]:
# Get list of all tables
tables_query = """
SELECT table_name, table_type
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

tables_df = pd.read_sql(tables_query, pd_engine)
print("Available tables:")
print(tables_df.to_string(index=False))

In [ ]:
# Display schema for all tables
for table in tables_df['table_name']:
    print(f"\n{'='*60}")
    print(f"Schema for table: {table}")
    print(f"{'='*60}")
    schema_df = explore_table_schema(table)
    print(schema_df.to_string(index=False))

    # Get row count
    count_query = f"SELECT COUNT(*) as row_count FROM {table};"
    count_df = pd.read_sql(count_query, pd_engine)
    print(f"\nRow count: {count_df['row_count'].iloc[0]:,}")

## Data Quality Analysis

Now let's create functions to identify data quality issues like missing fields, duplicates, and anomalies.

In [ ]:
# Analyze data quality for all tables
data_quality_results = {}

for table in tables_df['table_name']:
    try:
        print(f"\n{'#'*60}")
        print(f"Analyzing table: {table}")
        print(f"{'#'*60}")

        # Missing data analysis
        df, missing_stats = analyze_missing_data(table)

        if df is not None:
            # Duplicate analysis
            duplicates = find_duplicates(table)

            data_quality_results[table] = {
                'dataframe': df,
                'missing_stats': missing_stats,
                'duplicates': duplicates
            }
        else:
            data_quality_results[table] = {
                'dataframe': None,
                'missing_stats': None,
                'duplicates': None
            }

    except Exception as e:
        print(f"Error analyzing {table}: {str(e)}")
        continue

## Company-Specific Analysis

Let's analyze companies and their filing data using the utility functions.

In [ ]:
# Generate data quality summary
print("Data Quality Summary:")
print("="*40)
quality_summary = get_data_quality_summary()
print(quality_summary)

In [ ]:
# Run companies-specific analysis
companies_analysis = analyze_companies_data_quality()

In [ ]:
# Analyze filing patterns
filing_analysis = analyze_filing_patterns()

In [ ]:
# Example: Quick data exploration
# Look at companies missing summaries
missing_summaries = get_companies_with_missing_fields('summary')
print(f"Companies missing summaries: {len(missing_summaries)}")
if not missing_summaries.empty:
    print("\nSample companies missing summaries:")
    print(missing_summaries[['tickers', 'display_name']].head(3).to_string(index=False))

In [ ]:
# Get filing, document, aggregate counts, and summary status for companies missing summaries
if not missing_summaries.empty:
    print("\nFiling, Document, Aggregate Counts, and Summary Status for Companies Missing Summaries:")
    print("="*90)

    # Convert company IDs to a list of strings
    company_ids = [str(company_id) for company_id in missing_summaries['id'].tolist()]

    # Use the utility function to get filing, document, aggregate counts, and summary status
    filing_doc_counts = get_counts_for_companies(company_ids)

    if not filing_doc_counts.empty:
        # Display summary statistics
        print(f"Total companies missing summaries: {len(filing_doc_counts)}")
        print(f"Companies with filings: {len(filing_doc_counts[filing_doc_counts['filing_count'] > 0])}")
        print(f"Companies with documents: {len(filing_doc_counts[filing_doc_counts['document_count'] > 0])}")
        print(f"Companies with aggregates: {len(filing_doc_counts[filing_doc_counts['aggregate_count'] > 0])}")
        print(f"Companies with summaries: {len(filing_doc_counts[filing_doc_counts['has_summary'] == 'Yes'])}")
        print(f"Companies with both filings and documents: {len(filing_doc_counts[(filing_doc_counts['filing_count'] > 0) & (filing_doc_counts['document_count'] > 0)])}")
        print(f"Companies with filings, documents, and aggregates: {len(filing_doc_counts[(filing_doc_counts['filing_count'] > 0) & (filing_doc_counts['document_count'] > 0) & (filing_doc_counts['aggregate_count'] > 0)])}")

        # Show companies with filings but no documents (potential data issues)
        filings_no_docs = filing_doc_counts[(filing_doc_counts['filing_count'] > 0) & (filing_doc_counts['document_count'] == 0)]
        if not filings_no_docs.empty:
            print(f"\nCompanies with filings but no documents ({len(filings_no_docs)} companies):")
            print(filings_no_docs[['display_name', 'has_summary', 'filing_count', 'aggregate_count', 'latest_filing_date']].to_string(index=False))

        # Show companies with documents but no aggregates (potential opportunities)
        docs_no_aggregates = filing_doc_counts[(filing_doc_counts['document_count'] > 0) & (filing_doc_counts['aggregate_count'] == 0)]
        if not docs_no_aggregates.empty:
            print(f"\nCompanies with documents but no aggregates ({len(docs_no_aggregates)} companies):")
            print(docs_no_aggregates[['display_name', 'has_summary', 'document_count', 'filing_count', 'latest_filing_date']].head(3).to_string(index=False))
            if len(docs_no_aggregates) > 3:
                print(f"... and {len(docs_no_aggregates) - 3} more companies")

        # Show companies with aggregates but no summaries (prime candidates for summary generation)
        aggregates_no_summaries = filing_doc_counts[(filing_doc_counts['aggregate_count'] > 0) & (filing_doc_counts['has_summary'] == 'No')]
        if not aggregates_no_summaries.empty:
            print(f"\nCompanies with aggregates but no summaries - Prime candidates for summary generation ({len(aggregates_no_summaries)} companies):")
            print(aggregates_no_summaries[['display_name', 'aggregate_count', 'document_count', 'filing_count', 'latest_filing_date']].head(3).to_string(index=False))
            if len(aggregates_no_summaries) > 3:
                print(f"... and {len(aggregates_no_summaries) - 3} more companies")

        # Show companies with no filings, documents, or aggregates
        no_data = filing_doc_counts[(filing_doc_counts['filing_count'] == 0) & (filing_doc_counts['document_count'] == 0) & (filing_doc_counts['aggregate_count'] == 0)]
        if not no_data.empty:
            print(f"\nCompanies with no filings, documents, or aggregates ({len(no_data)} companies):")
            print(no_data[['display_name', 'tickers', 'sic_description']].head(3).to_string(index=False))
            if len(no_data) > 3:
                print(f"... and {len(no_data) - 3} more companies")

        # Show aggregate summary statistics
        total_aggregates = filing_doc_counts['aggregate_count'].sum()
        avg_aggregates_per_company = filing_doc_counts['aggregate_count'].mean()
        max_aggregates = filing_doc_counts['aggregate_count'].max()

        print(f"\nAggregate Summary:")
        print(f"Total aggregates across all companies: {total_aggregates}")

        # Summary status breakdown
        summary_counts = filing_doc_counts['has_summary'].value_counts()
        print(f"\nSummary Status Breakdown:")
        for status, count in summary_counts.items():
            percentage = (count / len(filing_doc_counts)) * 30
            print(f"Companies with summary = {status}: {count} ({percentage:.1f}%)")

    else:
        print("No results found for the query.")
else:
    print("No companies missing summaries found.")

In [ ]:
# count the average length of document for the companies above ^

In [ ]:
dc =filing_doc_counts
dc = dc[["tickers", "has_summary", "aggregate_count", "filing_count", "document_count"]]

dc.head()


In [ ]:
tickers_list = dc['tickers'].tolist()
tickers_list = [a[0] for a in tickers_list]
print(tickers_list)

In [ ]:
# Get companies that DO have summaries
companies_with_summaries_query = """
SELECT
    id,
    cik,
    name,
    display_name,
    tickers,
    summary
FROM companies
WHERE summary IS NOT NULL;
"""

companies_with_summaries = pd.read_sql(companies_with_summaries_query, pd_engine)

tickers_list = companies_with_summaries["tickers"].tolist()
tickers_list = [a[0] for a in tickers_list]
print(len(tickers_list))
print(tickers_list)